Figure des potentiels essaies à supprimer - aide pour définir des critères d'élimination

In [1]:
%load_ext autoreload
%autoreload 2
from __future__ import division, print_function, absolute_import
import pickle
import os
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
from edfreader import read_edf
import numpy as np
from lmfit import  Model, Parameters
%matplotlib inline

In [2]:
def exponentiel (x, tau, maxi, start_anti, v_anti, latence, bino) :
    '''
    tau -- courbe
    maxi -- maximum
    latence -- tps où commence le mvt
    bino -- binomial
    
    start_anti = debut de l'anticipation
    v_anti =  vitesse de l'anticipation
    ''' 
    v_anti = v_anti/1000 # pour passer de sec à ms
    time = np.arange(len(x))
    vitesse = []
                
    for t in range(len(time)):
        
        if start_anti >= latence :
            if time[t] < latence :
                vitesse.append(0)
            else :
                vitesse.append((bino*2-1)*maxi*(1-np.exp(-1/tau*(time[t]-latence))))
        else :

            if time[t] < start_anti :
                vitesse.append(0)
            else :
                if time[t] < latence :
                    #vitesse.append((bino*2-1)*(time[t]-start_anti)*v_anti)
                    vitesse.append((time[t]-start_anti)*v_anti)
                    x = (time[t]-start_anti)*v_anti
                else :
                    vitesse.append((bino*2-1)*maxi*(1-np.exp(-1/tau*(time[t]-latence)))+x)
    return vitesse

In [3]:
!ls ../data/*.asc

../data/enregistrement_AM_2017-10-23_100057.asc
../data/enregistrement_BMC_2017-09-26_095637.asc
../data/enregistrement_CS_2017-10-03_134421.asc
../data/enregistrement_DC_2017-09-27_161040.asc
../data/enregistrement_FM_2017-10-03_143803.asc
../data/enregistrement_IP_2017-09-28_115250.asc
../data/enregistrement_LB_2017-09-20_151043 (1).asc
../data/enregistrement_LB_2017-09-20_151043.asc
../data/enregistrement_OP_2017-10-26_121823.asc
../data/enregistrement_RS_2017-11-08_094717.asc
../data/enregistrement_SR_2017-11-16_153313.asc
../data/enregistrement_TN_2017-11-08_150410.asc
../data/enregistrement_YK_2017-11-17_172706.asc


In [4]:
timeStr = '2017-09-28_115250'
observer = 'IP'
mode = 'enregistrement'

In [5]:
file = os.path.join('..','data', mode + '_' + observer + '_' + timeStr + '.pkl')
with open(file, 'rb') as fichier :
    exp = pickle.load(fichier, encoding='latin1')
      
resultats = os.path.join('..','data', mode + '_' + observer + '_' + timeStr + '.asc')
data = read_edf(resultats, 'TRIALID')

In [6]:
N_trials = exp['N_trials']
N_blocks = exp['N_blocks']
screen_width_px = exp['screen_width_px']
screen_height_px = exp['screen_height_px']
V_X = exp['V_X']
RashBass = exp['RashBass']
stim_tau = exp['stim_tau']
p = exp['p']

In [7]:
for block in range(N_blocks) :
    if observer == 'BMC' :
        if block == 0 :
            liste_trial = [156]
        elif block == 1 :
            liste_trial = [129,131,136,137]
        elif block == 2 :
            liste_trial = [79,137,145]

    if observer == 'CS' :
        if block == 0 :
            liste_trial = [96,162,175]
        elif block == 1 :
            liste_trial = [43,45,53,58,85,119,122,123,124,125,126,127,135,138,141,163,167,195]
        elif block == 2 :
            liste_trial = [7,8,11,12,13,14,15,25,36,37,87,90,91,95,138,166,174]

    if observer == 'DC' :
        if block == 0 :
            liste_trial = [47,57,65,66,68,70,73,74,79,80,81,101,131,161,163,166,169,170,172,173,176,177]
        elif block == 1 :
            liste_trial = [8,33,55,64,65,76,77,78,90,104,106,109,111,120,123,124,131,132,155]
        elif block == 2 :
            liste_trial = [7,17,18,31,36,44,50,59,67,78,83]

    if observer == 'FM' :
        if block == 0 :
            liste_trial = [13,29,50,58,59,63,73,76,77,78,85,89,104,107,120,129,130,131,132,141,142,153,154,155,156,159,160,162,163,164,165,167,169,170,171,174,176,177,178]
        elif block == 1 :
            liste_trial = [12,31,54,55,64,65,66,70,77,79,90,96,98,99,111,117]
        elif block == 2 :
            liste_trial = [79,100,119,133,136,139,141,142,143,144,145,146,148,155,156,159,160,161,162,163,165,169,171,173,175,177,179,184,185,186,188]

    if observer == 'IP':
        if block == 0 :
            liste_trial = [124,168]
        elif block == 1 :
            liste_trial = [1,39,78,153]
        elif block == 2 :
            liste_trial = [47,115]
    
    
    fig_width= 12
    fig, axs = plt.subplots(len(liste_trial), 1, figsize=(fig_width, (fig_width*(len(liste_trial)/2))/1.6180))
    
    x = 0
    for trial in liste_trial :
        
        if len(liste_trial)==1 :
            ax = axs
        else :
            ax = axs[x]
        
        print(block, trial)
        trial_data = trial + N_trials*block
        data_x = data[trial_data]['x']
        data_y = data[trial_data]['y']
        trackertime = data[trial_data]['trackertime']

        StimulusOn = data[trial_data]['events']['msg'][10][0]
        StimulusOf = data[trial_data]['events']['msg'][14][0]
        TargetOn = data[trial_data]['events']['msg'][15][0]
        TargetOff = data[trial_data]['events']['msg'][16][0]
        saccades = data[trial_data]['events']['Esac']
        
        trackertime_0 = data[trial_data]['trackertime'][0]

        gradient_x = np.gradient(data_x)
        gradient_deg = gradient_x * 1/exp['px_per_deg'] * 1000 # gradient en deg/sec

        ##################################################
        # SUPPRESSION DES SACCADES
        ##################################################
        gradient_deg_NAN = gradient_deg
        for s in range(len(saccades)) :
            if saccades[s][1]-trackertime_0+15 <= (len(trackertime)) :
                for x_data in np.arange((saccades[s][0]-trackertime_0-5), (saccades[s][1]-trackertime_0+15)) :
                    gradient_deg_NAN[x_data] = np.nan
            else :
                for x_data in np.arange((saccades[s][0]-trackertime_0-5), (len(trackertime))) :
                    gradient_deg_NAN[x_data] = np.nan
            
        stop_latence = []    
        for s in range(len(saccades)) :
            if (saccades[s][0]-trackertime_0) >= (TargetOn-trackertime_0+100) :
                stop_latence.append((saccades[s][0]-trackertime_0))
        if stop_latence==[] :
            stop_latence.append(len(trackertime))
        ##################################################
        
        start = TargetOn
        
        StimulusOn_s = StimulusOn - start
        StimulusOf_s = StimulusOf - start
        TargetOn_s = TargetOn - start
        TargetOff_s = TargetOff - start
        trackertime_s = trackertime - start
        ##################################################
        # FIT
        ##################################################
        model = Model(exponentiel)
        bino=p[trial, block, 0]
        params = Parameters()
        params.add('tau', value=15., min=13., max=70.)#, vary=False)
        params.add('maxi', value=15., min=1., max=40.)#, vary=False)
        params.add('latence', value=TargetOn-trackertime_0+100, min=TargetOn-trackertime_0+75, max=stop_latence[0])
        params.add('start_anti', value=TargetOn-trackertime_0-100, min=StimulusOf-trackertime_0-200, max=TargetOn-trackertime_0+75)
        params.add('v_anti', value=0., min=-40., max=40.)
        params.add('bino', value=bino, min=0, max=1, vary=False)

        result_deg = model.fit(gradient_deg_NAN[:-280], params, x=trackertime[:-280], fit_kws={'nan_policy': 'omit'})
        ##################################################

        ax.cla() # pour remettre ax figure a zero
        ax.axis([StimulusOn_s-10, TargetOff_s+10, -40, 40])

        ax.plot(trackertime_s, gradient_deg_NAN, color='k', alpha=0.6)
        ax.plot(trackertime_s[:-280], result_deg.init_fit, 'r--', linewidth=2)
        ax.plot(trackertime_s[:-280], result_deg.best_fit, color='r', linewidth=2)
        ax.plot(trackertime_s, np.ones(np.shape(trackertime_s)[0])*(bino*2-1)*(15), color='k', linewidth=0.2, alpha=0.2)
        ax.plot(trackertime_s, np.ones(np.shape(trackertime_s)[0])*(bino*2-1)*(10), color='k', linewidth=0.2, alpha=0.2)
        ax.axvspan(StimulusOn_s, StimulusOf_s, color='k', alpha=0.2)
        ax.axvspan(StimulusOf_s, TargetOn_s, color='r', alpha=0.2)
        ax.axvspan(TargetOn_s, TargetOff_s, color='k', alpha=0.15)
        for s in range(len(saccades)) :
            ax.axvspan(saccades[s][0]-start, saccades[s][1]-start, color='k', alpha=0.2)

        debut  = TargetOn - trackertime_0 # TargetOn - temps_0
        
        start_anti = result_deg.values['start_anti']-debut
        v_anti = result_deg.values['v_anti']
        latence = result_deg.values['latence']-debut
        tau = result_deg.values['tau']
        maxi = result_deg.values['maxi']

        ax.bar(latence, 80, bottom=-40, color='r', width=6, linewidth=0)
        
        if x==0 :
            ax.text(StimulusOn_s+(StimulusOf_s-StimulusOn_s)/2, 31, "FIXATION", color='k', fontsize=16, ha='center', va='bottom')
            ax.text(StimulusOf_s+(TargetOn_s-StimulusOf_s)/2, 31, "GAP", color='r', fontsize=16, ha='center', va='bottom')
            ax.text(TargetOn_s+(TargetOff_s-TargetOn_s)/2, 31, "POURSUITE", color='k', fontsize=16, ha='center', va='bottom')
            ax.text(latence+25, -35, "Latence"%(latence), color='r', fontsize=14)#,  weight='bold')
        ax.text(StimulusOn_s+15, 18, "start_anti: %s \nv_anti: %s"%(start_anti, v_anti), color='k', fontsize=14, va='bottom')
        ax.text(StimulusOn_s+15, -18, "latence: %s \ntau: %s \nmaxi: %s"%(latence, tau, maxi), color='k', fontsize=14, va='top')

        ax.set_xlabel('Time (ms)', fontsize=9)
        ax.set_ylabel(trial+1, fontsize=30)
        x=x+1
        
    plt.tight_layout() # pour supprimer les marge trop grande
    plt.subplots_adjust(hspace=0) # pour enlever espace entre les figures

    plt.savefig('figures/suppresion_potentiel/suppresion_potentiel_%s_%s.pdf'%(observer, block+1))
    plt.close()

print('FIN !!!')

0 124
0 168
1 1
1 39
1 78
1 153
2 47
2 115
FIN !!!
